In [43]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [44]:

data = pd.read_pickle('../create_SL_data/data_6months_levels_ua.pkl')

data.dropna(inplace=True)

data

,patient_number,cluster_0_size-diff%_pre_to_3h,cluster_1_size-diff%_pre_to_3h,cluster_2_size-diff%_pre_to_3h,cluster_3_size-diff%_pre_to_3h,cluster_4_size-diff%_pre_to_3h,cluster_5_size-diff%_pre_to_3h,cluster_6_size-diff%_pre_to_3h,cluster_7_size-diff%_pre_to_3h,cluster_8_size-diff%_pre_to_3h,...,q90_difference_pre_7d_cluster_19_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_20_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_21_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_22_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_23_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_24_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_25_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_26_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_27_158Gd_pSTAT3_Y705,BCRABL
0,01,112.311228,4.657895,193.583600,17.357513,131.736527,41.726700,-9.016142,581.484794,341.481481,...,-0.036379,0.538215,0.244545,0.173798,0.520689,0.000000,0.445879,0.528887,-0.634862,0.0320
1,02,-39.227196,-35.480393,-44.449358,-42.916013,-23.653129,-27.315147,-71.392762,-4.200082,-66.785333,...,0.409531,0.471072,0.520922,0.195533,0.814652,0.000000,0.649210,0.523819,-0.824650,0.0075
2,03,107.606356,226.932177,32.365967,344.897484,21.987033,155.866808,96.242718,139.396945,20.366891,...,0.033095,-0.010136,0.182196,0.013442,0.234621,0.450780,0.509549,0.256670,-2.014077,0.0760
3,04,-25.682913,124.070561,135.671642,44.396807,-1.134931,83.993576,2.053080,36.713996,-25.974026,...,0.197905,-0.030211,0.115321,0.107060,0.578006,0.125471,0.312356,0.236907,-1.469164,0.6000
4,05,201.201479,43.747082,122.874251,43.729788,-36.776933,182.597569,-3.854390,119.651741,6.158358,...,0.528722,0.116360,0.030627,0.588062,0.544317,0.414612,0.000000,0.000000,-0.530531,0.6300
5,06,1.924515,15.800274,30.471903,-24.722838,-0.323767,83.695906,-34.709775,-28.399925,50.686604,...,0.228519,-0.704528,0.353815,0.126909,0.615345,0.805617,0.184870,0.000000,-1.011029,3.3000
6,07,-8.081530,-18.147677,66.870229,-41.526781,119.244513,-4.668262,-14.515928,-0.029555,125.590251,...,0.206264,0.310430,0.432708,0.101798,0.122739,-0.723297,0.275844,-0.836546,-0.386960,0.0054
7,08,-31.193715,-35.195035,25.087948,-39.439962,-17.761805,-27.302254,-34.826649,-12.109273,-26.896508,...,0.085245,-0.113622,0.375774,0.155500,-0.032117,0.751987,0.243146,0.494612,-1.508415,0.1200
8,09,12.781223,-20.827075,54.025045,-10.660076,100.369004,-8.090675,-24.769019,-14.171028,209.620596,...,0.838215,0.285612,0.294480,0.515315,0.613103,1.708393,-0.129841,0.539467,-1.008428,0.0350
9,11,-20.563531,27.474541,24.549279,-6.790533,175.652174,2.324571,6.422837,-6.846860,289.677419,...,0.683557,-0.295606,0.232018,0.329252,0.860805,0.000000,-0.124076,-0.068443,-0.809707,0.0000


In [45]:
data['patient_number'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '11', '12',
       '13', '14', '15', '17', '18', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '32', '33', '34', '35', '36', '37', '38'],
      dtype=object)

In [46]:
X = data.drop(['BCRABL', 'patient_number'], axis=1)  
y = data['BCRABL']                                 

patient_numbers = data['patient_number']

## Leave one out nested cross validation

In [132]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import mean_squared_error

models = {
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 10, 50]}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
    },
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}  
    }
}

feature_importances = {name: pd.DataFrame(np.zeros((X.shape[0], X.shape[1])), columns=X.columns) for name in models if name == 'RandomForest'}
scores = {name: {'mse_scores': [], 'rmse_scores': [], 'best_params': []} for name in models}

outer_cv = LeaveOneOut()
inner_cv = LeaveOneOut()

for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, config in models.items():
        if config['params']:
            grid_search = GridSearchCV(config['model'], config['params'], cv=inner_cv, scoring='neg_mean_absolut_error')
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            scores[name]['best_params'].append(grid_search.best_params_)
        else:
            best_model = config['model']
            best_model.fit(X_train, y_train)
            scores[name]['best_params'].append({})

        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        scores[name]['mse_scores'].append(mse)
        scores[name]['rmse_scores'].append(rmse)

        if name == 'RandomForest':
            feature_importances[name].iloc[fold_idx, :] = best_model.feature_importances_

    print(f'Fold {fold_idx + 1} completed')

results = {}
for name in models:
    average_mse = np.mean(scores[name]['mse_scores'])
    average_rmse = np.mean(scores[name]['rmse_scores'])
    results[name] = {
        'Average MSE': average_mse,
        'Average RMSE': average_rmse,
        'Best Parameters': scores[name]['best_params'][-1]
    }

    if name == 'RandomForest':
        mean_fi = feature_importances[name].mean(axis=0).sort_values(ascending=False)
        results[name]['Feature Importances'] = mean_fi

for name, result in results.items():
    print(f"{name} Results:")
    for key, value in result.items():
        print(f"{key}: {value}")


Fold 1 completed
Fold 2 completed
Fold 3 completed
Fold 4 completed
Fold 5 completed
Fold 6 completed
Fold 7 completed
Fold 8 completed


KeyboardInterrupt: 

In [47]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

model = KNeighborsRegressor(n_neighbors=10, weights='uniform')
model1 = RandomForestRegressor(n_estimators=200, max_depth=3)

model.fit(X, y)
model1.fit(X, y)

RandomForestRegressor(max_depth=3, n_estimators=200)

In [48]:
y.describe()

count    33.000000
mean      0.414785
std       1.211437
min       0.000000
25%       0.010000
50%       0.040000
75%       0.130000
max       6.320000
Name: BCRABL, dtype: float64

In [49]:
model.score(X, y)

0.026947051782605036

In [50]:
model1.score(X, y)

0.855332221547836

In [51]:
data_pool2 = pd.read_pickle('../create_SL_data_pool2/data_6months_levels_ua_pool2.pkl')

pool2_patients = pd.DataFrame(data_pool2['patient_number'])



In [52]:
X_test = data_pool2.drop(['BCRABL', 'patient_number'], axis=1)  
y_test = data_pool2['BCRABL']                                  

y_test

0    0.0200
1    2.1100
2    0.0020
3    1.0400
4    0.2220
5    0.0100
6    5.4000
7    0.0440
8    0.0026
Name: BCRABL, dtype: float64

In [53]:
model1.predict(X_test)


array([0.31898571, 0.40184195, 0.51171284, 1.18639591, 0.37899212,
       0.54027263, 0.54131169, 0.61366752, 0.5343602 ])

In [59]:
pd.set_option('display.max_colwidth', None)

coefficients = model1.feature_importances_  

feature_importance = pd.DataFrame(data={'Feature': X.columns, 'Importance': coefficients})

feature_importance['Absolute Importance'] = feature_importance['Importance'].abs()
feature_importance = feature_importance.sort_values(by='Absolute Importance', ascending=False)

feature_importance.head(30)

,Feature,Importance,Absolute Importance
711,90th_159Tb_pMAPKAPK T334_3h_cluster_11,0.048494,0.048494
1425,q90_difference_pre_3h_cluster_25_151Eu_pSTAT3_S727,0.035611,0.035611
186,90th_151Eu_pSTAT3 S727_pre_cluster_18,0.035451,0.035451
585,90th_150Nd_pSTAT5 Y694_3h_cluster_25,0.027111,0.027111
539,90th_144Nd_pTyr_3h_cluster_7,0.026557,0.026557
858,90th_173Yb_STAT3tot_3h_cluster_18,0.022470,0.022470
214,90th_153Eu_pSTAT1 Y701_pre_cluster_18,0.022189,0.022189
769,90th_170Er_pSRC Y418_3h_cluster_13,0.021949,0.021949
688,90th_158Gd_pSTAT3 Y705_3h_cluster_16,0.019909,0.019909
485,90th_142Nd_cCaspase3_3h_cluster_9,0.018034,0.018034


In [54]:
pool2_patients['pred'] = model1.predict(X_test)
pool2_patients['month'] = 6
pool2_patients['patient_number'] = pool2_patients['patient_number'].astype(int)

pool2_patients

,patient_number,pred,month
0,39,0.318986,6
1,40,0.401842,6
2,41,0.511713,6
3,42,1.186396,6
4,44,0.378992,6
5,45,0.540273,6
6,54,0.541312,6
7,56,0.613668,6
8,57,0.534360,6


In [55]:
patient_numbers_pool2 = data_pool2['patient_number'].astype(int)

response = pd.read_csv('../response/responses_all.csv')
response_pool2 = response[response['patient_number'].isin(patient_numbers_pool2)]

response_pool2

,patient_number,patient_id,batch,month,BCRABL
237,39,4810_00009,5,3,0.070
238,39,4810_00009,5,6,0.020
239,39,4810_00009,5,9,0.020
240,39,4810_00009,5,12,0.007
241,39,4810_00009,5,15,0.009
...,...,...,...,...,...
295,57,4306_00004,6,9,0.025
296,57,4306_00004,6,12,NaN
297,57,4306_00004,6,15,NaN
298,57,4306_00004,6,17,0.007


In [56]:
merged_data = pd.merge(response_pool2, pool2_patients, on=['patient_number', 'month'], how='outer')
merged_data

,patient_number,patient_id,batch,month,BCRABL,pred
0,39,4810_00009,5,3,0.070,NaN
1,39,4810_00009,5,6,0.020,0.318986
2,39,4810_00009,5,9,0.020,NaN
3,39,4810_00009,5,12,0.007,NaN
4,39,4810_00009,5,15,0.009,NaN
...,...,...,...,...,...,...
58,57,4306_00004,6,9,0.025,NaN
59,57,4306_00004,6,12,NaN,NaN
60,57,4306_00004,6,15,NaN,NaN
61,57,4306_00004,6,17,0.007,NaN


In [57]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly 
fig = go.Figure()

for idx, patient in enumerate(merged_data['patient_number'].unique()):
    patient_data = merged_data[merged_data['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color) 
    ))
    
  
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=1)

fig.show()



## Creating the same plots for training data

In [36]:
pool1_patients = pd.DataFrame(data['patient_number'])
pool1_patients['patient_number'] = pool1_patients['patient_number'].astype(int)
pool1_patients['pred'] = model1.predict(X)
pool1_patients['month'] = 6

pool1_patients

,patient_number,pred,month
0,1,0.739067,6
1,2,0.555730,6
2,3,0.450887,6
3,4,0.530714,6
4,5,0.438253,6
5,6,2.217967,6
6,7,0.301393,6
7,8,0.176285,6
8,9,0.113264,6
9,11,0.105977,6


In [37]:
merged_data = pd.merge(response, pool1_patients, on=['patient_number', 'month'], how='outer')
merged_data

merged_data['BCRABL'] = merged_data['BCRABL'].replace(0, 0.001)

In [38]:
merged_data_1 = merged_data[merged_data['batch'] == 1]
merged_data_2 = merged_data[merged_data['batch'] == 2]
merged_data_3 = merged_data[merged_data['batch'] == 3]
merged_data_4 = merged_data[merged_data['batch'] == 4]

In [39]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly 

fig = go.Figure()

for idx, patient in enumerate(merged_data_1['patient_number'].unique()):
    patient_data = merged_data_1[merged_data_1['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False 
        ))
        
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False 
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values Batch 1',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=1)

fig.show()


In [40]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly 

fig = go.Figure()

for idx, patient in enumerate(merged_data_2['patient_number'].unique()):
    patient_data = merged_data_2[merged_data_2['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10), 
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values Batch 2',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=1)

fig.show()


In [41]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data_3['patient_number'].unique()):
    patient_data = merged_data_3[merged_data_3['patient_number'] == patient]
    patient_color = colors[idx % len(colors)] 

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values Batch 3',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=1)

fig.show()


In [42]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly 

fig = go.Figure()

for idx, patient in enumerate(merged_data_4['patient_number'].unique()):
    patient_data = merged_data_4[merged_data_4['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color) 
    ))
    
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10), 
            showlegend=False  
        ))
        
       
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))


fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values Batch 4',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)


fig.add_hline(y=1)


fig.show()
